# Trabajo Práctico 2 : Críticas Cinematográficas

### Grupo 01 - Integrantes:
        Cabrera Rodríguez, Mateo 
        Gonzalez Alejo, Camila
        Bocanegra, Eduardo Martín

# Setup

### Cargar bibliotecas

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics

### Cargar dataset

In [2]:
ds = pd.read_csv("datasets/train.csv")
print(ds.head())

   ID                                          review_es sentimiento
0   0  Uno de los otros críticos ha mencionado que de...    positivo
1   1  Una pequeña pequeña producción.La técnica de f...    positivo
2   2  Pensé que esta era una manera maravillosa de p...    positivo
3   3  Básicamente, hay una familia donde un niño peq...    negativo
4   4  El "amor en el tiempo" de Petter Mattei es una...    positivo


In [3]:
reviews = ds["review_es"]
sentiments = [1 if sentiment == "positivo" else 0 for sentiment in ds["sentimiento"]]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(reviews, sentiments, test_size=0.2, random_state=42)

In [5]:
# Vectorizar las reviews
vectorizador = CountVectorizer()
X_train_v = vectorizador.fit_transform(X_train)
X_test_v = vectorizador.transform(X_test)


In [6]:
nb_model = BernoulliNB()
nb_model.fit(X_train_v, y_train)

BernoulliNB()

In [7]:
y_pred = nb_model.predict(X_test_v)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))
print("F1 Score:", metrics.f1_score(y_test, y_pred))

Accuracy: 0.8466
Precision: 0.8670157068062827
Recall: 0.8215915856320699
F1 Score: 0.8436926839209293


In [78]:
test_ds = pd.read_csv("datasets/test.csv")
test_ds["sentimiento"] = nb_model.predict(vectorizador.transform(test_ds["review_es"]))
# convert "sentimiento" to "positivo" or "negativo"
test_ds["sentimiento"] = test_ds["sentimiento"].apply(lambda x: "positivo" if x == 1 else "negativo")
test_ds[["ID","sentimiento"]].to_csv("datasets/binomialNB.csv", index=False)

In [15]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


def remove_special_characters(text):
    # Eliminar puntuaciones y caracteres especiales excepto letras y números
    text = re.sub(r"[^\w\s]", "", text)
    return text

def convert_to_lowercase(text):
    # Convertir el texto a minúsculas
    text = text.lower()
    return text

def tokenize_text(text):
    # Tokenización de palabras
    tokens = word_tokenize(text, language="spanish")
    return tokens

def remove_stop_words(tokens):
    # Eliminar palabras vacías
    stop_words = set(stopwords.words("spanish"))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens


def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\digix\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\digix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\digix\AppData\Roaming\nltk_data...


In [16]:
X_preprocessed = []

for review in reviews:
    review = remove_special_characters(review)
    review = convert_to_lowercase(review)
    tokens = tokenize_text(review)
    tokens = remove_stop_words(tokens)
    tokens = lemmatize_tokens(tokens)
    review = " ".join(tokens)
    X_preprocessed.append(review)

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, sentiments, test_size=0.2, random_state=42)

In [17]:
review_0 = reviews[0]
print("Original:", review_0)
print("remove_special_characters:", remove_special_characters(review_0))
print("convert_to_lowercase:", convert_to_lowercase(review_0))
print("tokenize_text:", tokenize_text(review_0))
print("remove_stop_words:", remove_stop_words(tokenize_text(review_0)))
print("lemmatize_tokens:", lemmatize_tokens(remove_stop_words(tokenize_text(review_0))))
print("X_preprocessed[0]:", X_preprocessed[0])

Original: Uno de los otros críticos ha mencionado que después de ver solo 1 Oz Episodio, estará enganchado. Tienen razón, ya que esto es exactamente lo que sucedió conmigo. La primera cosa que me golpeó sobre Oz fue su brutalidad y sus escenas de violencia inconfiadas, que se encuentran a la derecha de la palabra. Confía en mí, este no es un espectáculo para los débiles de corazón o tímido. Este espectáculo no extrae punzones con respecto a las drogas, el sexo o la violencia. Es Hardcore, en el uso clásico de la palabra. Se llama OZ, ya que es el apodo dado al Penitenciario del Estado de Seguridad Máximo de Oswald. Se centra principalmente en la ciudad de Emeralda, una sección experimental de la prisión donde todas las células tienen frentes de vidrio y se enfrentan hacia adentro, por lo que la privacidad no es alta en la agenda. Em City es el hogar de muchos ... Fariarios, musulmanes, gangstas, latinos, cristianos, italianos, irlandeses y más ... así que las esposas, las miradas de mu

In [22]:
# Vectorizar las reviews
vectorizador = CountVectorizer()
X_train_v = vectorizador.fit_transform(X_train)
X_test_v = vectorizador.transform(X_test)

In [19]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "alpha": [0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    "binarize": [0.0, 0.5, 1.0, 2.0, 5.0, 10.0],
    "fit_prior": [True, False]
}

nb_model = BernoulliNB()
grid_search = GridSearchCV(estimator=nb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_v, y_train)

print(grid_search.best_params_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
{'alpha': 1.0, 'binarize': 0.0, 'fit_prior': True}


In [20]:
best_params = {'alpha': 1.0, 'binarize': 0.0, 'fit_prior': True}

In [24]:
nb_model = BernoulliNB(**best_params)
nb_model.fit(X_train_v, y_train)

y_pred = nb_model.predict(X_test_v)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))
print("F1 Score:", metrics.f1_score(y_test, y_pred))

Accuracy: 0.8368
Precision: 0.8659505907626208
Recall: 0.7999603095852351
F1 Score: 0.8316484423354652


In [25]:
test_ds = pd.read_csv("datasets/test.csv")
test_ds["sentimiento"] = nb_model.predict(vectorizador.transform(test_ds["review_es"]))
# convert "sentimiento" to "positivo" or "negativo"
test_ds["sentimiento"] = test_ds["sentimiento"].apply(lambda x: "positivo" if x == 1 else "negativo")
test_ds[["ID","sentimiento"]].to_csv("datasets/binomialNB_tokenizado.csv", index=False)

# Visualizacion de Datos

# Preprocesamiento de Datos

# Transformacion de Datos

# Exploracion de Datos

# Analisis de Datos